In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

In [2]:
df_geo_raw_s = pd.read_csv('U.S._Life_Expectancy_at_Birth_by_State_and_Census_Tract_-_2010-2015.csv', encoding='latin-1')

In [3]:
df_geo_raw_s_cleaned = df_geo_raw_s[~df_geo_raw.eq('(blank)').any(axis=1)]

In [4]:
df_geo_raw_s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73121 entries, 0 to 73120
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   State                        73121 non-null  object 
 1   County                       73121 non-null  object 
 2   CensusTract                  73070 non-null  float64
 3   LifeExpectancy               67199 non-null  float64
 4   LifeExpectancyRange          67199 non-null  object 
 5   LifeExpectancyStandardError  67199 non-null  float64
dtypes: float64(3), object(3)
memory usage: 3.3+ MB


In [5]:
print(df_geo_raw_s_cleaned.isnull().sum().sum())
print(df_geo_raw_s_cleaned.isnull().any())

17766
State                          False
County                         False
CensusTract                    False
LifeExpectancy                  True
LifeExpectancyRange             True
LifeExpectancyStandardError     True
dtype: bool


In [6]:
df_geo_raw_s_cleaned.head()

,State,County,CensusTract,LifeExpectancy,LifeExpectancyRange,LifeExpectancyStandardError
1,Alabama,"Autauga County, AL",201.0,73.1,56.9-75.1,2.2348
2,Alabama,"Autauga County, AL",202.0,76.9,75.2-77.5,3.3453
3,Alabama,"Autauga County, AL",203.0,NaN,NaN,NaN
4,Alabama,"Autauga County, AL",204.0,75.4,75.2-77.5,1.0216
5,Alabama,"Autauga County, AL",205.0,79.4,77.6-79.5,1.1768


In [7]:
df = df_geo_raw_s_cleaned.copy()
# Split the 'LifeExpectancyRange' column by the '-' delimiter
split_values = df['LifeExpectancyRange'].str.split('-')

# Assign the first part (index 0) to 'LifeExpectancyLow'
df['LifeExpectancyLow'] = split_values.str[0]

# Assign the second part (index 1) to 'LifeExpectancyHigh'
df['LifeExpectancyHigh'] = split_values.str[1]

# Convert the new columns to numeric (float)
# errors='coerce' will handle any missing or problematic values by turning them into NaN
df['LifeExpectancyLow'] = pd.to_numeric(df['LifeExpectancyLow'], errors='coerce')
df['LifeExpectancyHigh'] = pd.to_numeric(df['LifeExpectancyHigh'], errors='coerce')

# Display the DataFrame to see the new columns
print(df.head())

# Check the info to confirm the new columns and their types
print(df.info())

     State              County  CensusTract  LifeExpectancy  \
1  Alabama  Autauga County, AL        201.0            73.1   
2  Alabama  Autauga County, AL        202.0            76.9   
3  Alabama  Autauga County, AL        203.0             NaN   
4  Alabama  Autauga County, AL        204.0            75.4   
5  Alabama  Autauga County, AL        205.0            79.4   

  LifeExpectancyRange  LifeExpectancyStandardError  LifeExpectancyLow  \
1           56.9-75.1                       2.2348               56.9   
2           75.2-77.5                       3.3453               75.2   
3                 NaN                          NaN                NaN   
4           75.2-77.5                       1.0216               75.2   
5           77.6-79.5                       1.1768               77.6   

   LifeExpectancyHigh  
1                75.1  
2                77.5  
3                 NaN  
4                77.5  
5                79.5  
<class 'pandas.core.frame.DataFrame'>
